<a href="https://colab.research.google.com/github/Ayavie/Speed-dating-match-prediction/blob/main/Speed_Dating_Match_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#✔️ Problem Formulation:
Define the problem. What is the input? What is the output? What data mining function is required? What could be the challenges? What is the impact? What is an ideal solution?

The objective of this problem to find whether 2 people matches in a certain speed dating program.

The input we have are the profile of each person who joined this program of some personal information about life style, education and job, preferences and so on. 

The output whether 2 people matches or not at the end of the program.

The challenge is to find the best way to deal with the missing values as they are alot.

The impact will be setting 2 people who are predicted to be matching together by the model and have higher success rate for the dating sessions program if the model has high accuracy.

I think the ideal solution could be a model predicting more than 70% evaluation metric.

#Importing the libraries and investigating the data

In [ ]:
#Importing Libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost.sklearn import XGBClassifier

In [ ]:
#Reading data files

#Training data
#Initialize main dataframe 
df_main=pd.read_csv('train.csv')

#Make a copy of the main dataframe for usage
df=df_main.copy()

df2=df_main.copy()
#Testing data
df_test=pd.read_csv('test.csv')

In [ ]:
# Test dataframe size
df_test.shape

(2469, 191)

In [ ]:
# Training dataframe size
df.shape

(5909, 192)

In [ ]:
#Checking sum of missing values in each column and putting them into a dataframe to read them

df_null=pd.DataFrame(df.isnull().sum())

df_null

,0
gender,0
idg,0
condtn,0
wave,0
round,0
...,...
sinc5_3,4496
intel5_3,4496
fun5_3,4496
amb5_3,4496


In [ ]:
#summtion of nulls in each column

df.isnull().sum()

gender         0
idg            0
condtn         0
wave           0
round          0
            ... 
sinc5_3     4496
intel5_3    4496
fun5_3      4496
amb5_3      4496
id             0
Length: 192, dtype: int64

In [ ]:
df.isnull().count()

gender      5909
idg         5909
condtn      5909
wave        5909
round       5909
            ... 
sinc5_3     5909
intel5_3    5909
fun5_3      5909
amb5_3      5909
id          5909
Length: 192, dtype: int64

In [ ]:
#Get null percentage in each column and convert it to dataframe 
null_percent = pd.DataFrame(((df.isnull().sum()/df.isnull().count()).sort_values(ascending=False))*100).T
null_percent

,num_in_3,numdat_3,expnum,amb7_2,sinc7_2,shar7_2,fun7_2,intel7_2,attr7_2,attr7_3,...,idg,samerace,match,partner,order,position,round,wave,condtn,id
0,92.215265,82.061262,78.304282,76.476561,76.476561,76.239634,76.121171,76.121171,76.121171,76.087324,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#df[colname]
null_percent[null_percent.columns[0]]

0    92.215265
Name: num_in_3, dtype: float64

In [ ]:
print(null_percent[null_percent.columns[1]]>=60)

0    True
Name: numdat_3, dtype: bool


In [ ]:
#Initialize a list and append column names that contain nulls more than 60% of its size 
null_cols=[]
for i in range(null_percent.shape[1]):
  if (null_percent[null_percent.columns[i]]>=60).any():
     colname=null_percent.columns[i]
     null_cols.append(colname)

null_cols




['num_in_3',
 'numdat_3',
 'expnum',
 'amb7_2',
 'sinc7_2',
 'shar7_2',
 'fun7_2',
 'intel7_2',
 'attr7_2',
 'attr7_3',
 'sinc7_3',
 'intel7_3',
 'fun7_3',
 'amb7_3',
 'shar7_3',
 'shar2_3',
 'attr5_3',
 'sinc5_3',
 'intel5_3',
 'fun5_3',
 'amb5_3',
 'shar4_3',
 'fun4_3',
 'intel4_3',
 'sinc4_3',
 'attr4_3',
 'attr2_3',
 'sinc2_3',
 'intel2_3',
 'fun2_3',
 'amb2_3',
 'amb4_3',
 'mn_sat']

#Assigning dependent and independent variables

In [ ]:
#Assigning dependent and independent variables
y = df['match'] # lower case for vector
X = df.drop('match', axis=1) # upper case for matrix
print('original shape', X.shape, y.shape)

# now we can split the data into train and validation sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print('y_test', y_test)

original shape (5909, 191) (5909,)
y_test 3489    0
4909    0
1425    0
4743    0
439     0
       ..
5333    0
4310    0
653     0
429     0
1773    1
Name: match, Length: 1182, dtype: int64


# Extracting numeric features and categorical features names

In [ ]:
# for later use

# numeric features can be selected by: (based on the df2.info() output )
features_numeric = list(X_train.select_dtypes(include=['float64', 'int64']))

# categorical features can be selected by: (based on the df2.info() output )
features_categorical = list(X_train.select_dtypes(include=['object']))

print('numeric features:', features_numeric)
print('categorical features:', features_categorical)

numeric features: ['gender', 'idg', 'condtn', 'wave', 'round', 'position', 'positin1', 'order', 'partner', 'pid', 'int_corr', 'samerace', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'age', 'field_cd', 'race', 'imprace', 'imprelig', 'goal', 'date', 'go_out', 'career_c', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy', 'expnum', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1', 'attr4_1', 'sinc4_1', 'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1', 'fun3_1', 'intel3_1', 'amb3_1', 'attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1', 'attr', 'sinc', 'intel', 'fun', 'amb', 'shar', 'like', 'prob', 'met', 'match_es', 'attr1_s', 'sin

#Strategy used throughout the whole code
1) Defining a function that creates a pipeline that determines preprocessing and classifier


2) Try 2 different classifiers ( RandomForest and XGBoost Classifier )

3) Using the best one with its default parameters 

4) Applying 3 different tuning techniques on it

    1. Grid search 

    2. Random search 

    3. Bayesian search 

5) Try different preprocessing (dropping features and using different imputers)

6) The evaluation metric used is AUROC.

##First pipeline function initialization
In this pipeline we will:

1. Use all features.
2. Numeric data: 
 * Fill numeric features than contain nans with the mean.
 * Standardize numeric data.
3. Categorical features:
  * Fill categorical features that contain nans with "missing_value".
  * One-Hot-Encode them.
4. Use RandomForest Classifier.
5. XGBoost Classifier.

**Thoughts:** Not much as this is the a base trial for the model. All I can expect is that there won't be high accuracy.










In [ ]:
# defining a function to drop columns determined before plus new ones which are considered useless or redundant 
def drop_features(df):
  df.drop(columns=null_cols+['id','from','career'],inplace=True)
  return df

In [ ]:
#Defining function for the whole pipeline used for the model (Preprocessing and model initialization)
def first_pipeline(df,classifier,imputer,all_features=True):
  
  def drop_features(df):
    df=df.drop(columns=null_cols,axis=1,inplace=True)
    return df 

  #Fixing random seed for reproducibility
  np.random.seed(0)
   

  # Define a pipe line for numeric feature preprocessing
  # We gave them a name so we can set their hyperparameters
  transformer_numeric = Pipeline(
    steps=[
        ('imputer', imputer),
        ('scaler', StandardScaler())]
)
  

  # Define a pipe line for categorical feature preprocessing
  # We gave them a name so we can set their hyperparameters
  # Filling missing values of categorical features with '0'
  transformer_categorical = Pipeline(
      steps=[
          ('imputer', SimpleImputer(strategy='constant')),
          ('onehot', OneHotEncoder(handle_unknown='ignore'))
      ]
  )
  print('done transformer_categorical ')
  # define the preprocessor 
  # we gave them a name so we can set their hyperparameters
  # we also specify what are the categorical 
  preprocessor = ColumnTransformer(
      transformers=[
          ('num', transformer_numeric, features_numeric),
          ('cat', transformer_categorical, features_categorical)
      ]
  )



  # combine the preprocessor with the model as a full tunable pipeline
  # we gave them a name so we can set their hyperparameters
  if (all_features==True):

    full_pipline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('my_classifier', 
            classifier,
            )
        ]
    )
    
  else:



        full_pipline = Pipeline(
        steps=[
            ('dropping_features',drop_features(df)),
            ('preprocessor', preprocessor),
            ('my_classifier', 
            classifier,
            )
        ]
    )
       


  return full_pipline

##**Next:** RandomForest

In [ ]:
# Calling pipeline function and assign it to model variable
model=first_pipeline(df,RandomForestClassifier(),SimpleImputer()).fit(X_train, y_train)
model.predict(X_test)


done transformer_categorical 


array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
#Make submission sample using predictions
submission = pd.DataFrame()

submission['id'] = df_test['id']

submission['match'] = model.predict_proba(df_test)[:,1]

submission.to_csv('submission_RF.csv', index=False)

### **Observation 1** : Random Forest
Got low testing accuracy of value $0.82959 !$

This means underfitting but not bad for starters with primary model only!



## **Next**: XGBoost
We'll try to use different optimizer which is XGBoost classifier with the same pipeline used.

### **Expectation**:
 We can not expect whether XGBoost will perform better using the same pipeline so let's find out the results!

In [ ]:
# Calling pipeline function and assign it to model variable
model=first_pipeline(df,XGBClassifier(),SimpleImputer()).fit(X_train, y_train)
model.predict(X_test)


done transformer_categorical 


array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
#Make submission sample using predictions
submission = pd.DataFrame()

submission['id'] = df_test['id']

submission['match'] = model.predict_proba(df_test)[:,1]

submission.to_csv('submission_xgb.csv', index=False)

### **Observation 2** : XGBoost 


XGBClassifier got $0.87092$ testing accuracy which is much higher than RandomForestClassifier!






## **Next**: 3 different tuning techniques

We'll use 3 different tuning techniques (GridSearch, RandomSearch, BayesianSearch) on XGBoostClassifier and try to get better accuracy.

### **Expectations** :

I can expect that the model's test accuracy will increase.

###  **GridSearch with XGBoost**

In [ ]:
# here we specify the search space
# `__` denotes an attribute of the preceeding name
# (e.g. my_classifier__n_estimators means the `n_estimators` param for `my_classifier`)
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    # preprocessor__num__imputer__strategy points to preprocessor->num (a Pipeline)-> imputer -> strategy
    'my_classifier__n_estimators': [10,15,20,25,30,35, 40],  
     # my_classifier__n_estimators points to my_classifier->n_estimators 
    'my_classifier__max_depth':[10,15, 20,25, 30]       
}

# cv=2 means two-fold cross-validation
# n_jobs means the cucurrent number of jobs
# (on colab since we only have two cpu cores, we set it to 2)
grid_search = GridSearchCV(
    model, param_grid, cv=4, verbose=1, n_jobs=2, 
    scoring='roc_auc')

grid_search.fit(X_train, y_train)

print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))

Fitting 4 folds for each of 70 candidates, totalling 280 fits
best score 0.8598146189936264
best score {'my_classifier__max_depth': 10, 'my_classifier__n_estimators': 40, 'preprocessor__num__imputer__strategy': 'median'}


In [ ]:
#Predicting testdata using gridseach with xgboost
grid_search.predict(X_test)

#Make submission sample using predictions
submission = pd.DataFrame()

submission['id'] = df_test['id']

submission['match'] = grid_search.predict_proba(df_test)[:,1]

submission.to_csv('submission_xgb_GS.csv', index=False)

GridSearch with XGBoost got testing accuracy score: **0.86287**

###  **RandomSearch with XGBoost**

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV


random_search = RandomizedSearchCV(
   model, param_grid, cv=4, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc',
    random_state=0)

random_search.fit(X_train, y_train)

print('best score {}'.format(random_search.best_score_))
print('best score {}'.format(random_search.best_params_))

Fitting 4 folds for each of 10 candidates, totalling 40 fits
best score 0.8557845752521942
best score {'preprocessor__num__imputer__strategy': 'median', 'my_classifier__n_estimators': 40, 'my_classifier__max_depth': 30}


In [ ]:
#Predicting testdata using gridseach with xgboost
random_search.predict(X_test)

#Make submission sample using predictions
submission = pd.DataFrame()

submission['id'] = df_test['id']

submission['match'] = random_search.predict_proba(df_test)[:,1]

submission.to_csv('submission_xgb_RS.csv', index=False)

RandomSearch with XGBoost got testing accuracy score: **0.85842**

###  **Bayesian Search with XGBoost**

In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 100 kB 4.8 MB/s 


In [ ]:
#Importing BayesSearch and fitting it with our model of XGBoost
from skopt import BayesSearchCV

bayes_search = BayesSearchCV(

model, param_grid, cv=4, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc',
    random_state=0
)

bayes_search.fit(X_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best score {}'.format(bayes_search.best_params_))


Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
best score 0.8569260235085413
best score OrderedDict([('my_classifier__max_depth', 10), ('my_classifier__n_estimators', 35), ('preprocessor__num__imputer__strategy', 'median')])


In [ ]:
#Predicting testdata using gridseach with xgboost
bayes_search.predict(X_test)

#Make submission sample using predictions
submission = pd.DataFrame()

submission['id'] = df_test['id']

submission['match'] = bayes_search.predict_proba(df_test)[:,1]

submission.to_csv('submission_xgb_BS.csv', index=False)

BayesSearch with XGBoost got testing accuracy score: **0.86076**

### **Observation 3:**



* GridSearch with XGBoost got training accuracy **0.85981** testing accuracy score: **0.86287**

* RandomSearch with XGBoost got training accuracy **0.85578** testing accuracy score: **0.85842**


* BayesSearch with XGBoost got training accuracy **0.85692** testing accuracy score: **0.86076**


> GridSearch got the highest score among the three of them but so far XGB classifier without gridsearch got higher accuracy of score $0.87092!!$

##Next We'll use different preprocessing technique
We'll drop columns that contain more than 60% null values with XGBoost and let's see how would that affect model accuracy!

In [ ]:
#drop_features(df2)

In [ ]:
# Calling pipeline function and assign it to model variable
model=first_pipeline(df2,XGBClassifier(),SimpleImputer(),all_features=False)
model.fit(X_train, y_train)
model.predict(X_test)


done transformer_categorical 


array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
#Make submission sample using predictions
submission = pd.DataFrame()

submission['id'] = df_test['id']

submission['match'] = model.predict_proba(df_test)[:,1]

submission.to_csv('submission_xgb2.csv', index=False)

#### **Observation 1** : Trial 2  
XGBClassifier got $0.87517$ testing accuracy which is slightly higher than last time with all features which got $0.8709$


## **Next** :
We'll use the previous model with iterative imputer instead of simple imputer

### **Expectation** :
 Higher accuracy as it estimates each feature from all the others.

In [ ]:
#Importing Terative Imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
# Calling pipeline function and assign it to model variable
model=first_pipeline(df2,XGBClassifier(),IterativeImputer(random_state=0))
model.fit(X_train, y_train)
model.predict(X_test)


done transformer_categorical 


array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
#Make submission sample using predictions
submission = pd.DataFrame()

submission['id'] = df_test['id']

submission['match'] = model.predict_proba(df_test)[:,1]

submission.to_csv('submission_xgb_itimp.csv', index=False)

#### **Observation 2** : Trial 2
And I guess I my expectation went wrong.
Using Iterative imputer got testing accuracy of value $0.87477$ which is slightly worse than using simple imputer which got testing accuracy $0.87517$
We can conclude that maybe depending on all features is not reliable and not all of them should contribute in predicting others which contain null values.


# **Final Observation** : 
Best model used was the one with feature selection(dropped some features) with XGB Classifier and Simple Imputer.

 (With header: $Observation 1: Trial 2$ ) 

#✔️ Answer the questions below (briefly):



🌈 1. Why a simple linear regression model (without any activation function) is not good for classification task, compared to Perceptron/Logistic regression?

> Answer: Linear Regression is not suitable for classification for 2 reasons. 
1. Linear Regression deals with continuous values whereas classification problems deals with discrete values.
2. The shift problem in threshold value when new data points are added, the line shifts to fit most of the data.

> On the other hand, Logistic Regression deals with discrete values unlike Linear Regression and it also maintains the value of the threshold even when new data points are added.

> Linear regression is the same as MLP with no activation function which is considered a closed form solution where you can compute the output directly and efficiently as well but once you add activation function to MLP it will be iterative solution in which the algorithm need to go through steps to compute the output and also computations to improve with each step and that would be a lot slower than direct solution.

> Another different is that in MLP with hidden layers and activation functions, the model becomes much more powerful and also has the ability to tune as when you increase number of hidden layers the model will be able to extract more features, get better accuracy and less error but in simple neural network or linear regression the model can only model linear function which are almost does not exist in real life senarios. 

🌈What's a decision tree and how it is different to a logistic regression model?

> Answer: Decision tree is an algorithm used for both regression and classification used to split dataset into smaller subsets until each leaf contain values belong to one only feature or category so it can not be split anymore.
Logistic regression is an algorithm used  for modeling the probability of a discrete outcome given an input variable. It is used in binary classification.

> Decision Trees bisect the space into smaller and smaller regions, while Logistic Regression fits a single line to divide the space exactly into two. For higher-dimensional data, these lines would generalize to planes and hyperplanes. A single linear boundary can sometimes be limiting for Logistic Regression.


🌈What's the difference between grid search and random search?

> Answer: GridSearch tries all combination of hyperparaeters range specfied to it. It allow you to know the best combination among them and best score. 
On the other hand, random search is tries random combinations of hyperparameters used in the search space and it allow you to know the best combination among the only trials it made and its best score.


🌈What's the difference between bayesian search and random search?


>Answer: Random search is tries random combinations of hyperparameters used in the search space and it allow you to know the best combination among the only trials it made and its best score. while Bayesian optimization is a sequential model-based optimization (SMBO) algorithm that uses the results from the previous iteration to decide the next hyperparameter value candidates.
So instead of blindly searching the hyperparameter space (like in grid search and random search), this method advocates the usage of intelligence to pick the next set of hyperparameters which will improve the model performance. We iteratively repeat this process until we converge to an optimum.


**References:**

https://medium.com/analytics-vidhya/why-linear-regression-is-not-suitable-for-classification-cd724dd61cb8

https://blog.bigml.com/2016/09/28/logistic-regression-versus-decision-trees/ 

https://medium.com/analytics-vidhya/comparison-of-hyperparameter-tuning-algorithms-grid-search-random-search-bayesian-optimization-5326aaef1bd1


https://cs.stackexchange.com/questions/28597/difference-between-multilayer-perceptron-and-linear-regression